# What is RAG?

## Defination

This is called retrieval augmented generation (RAG), as you would retrieve the relevant data and use it as augmented context for the LLM. Instead of relying solely on knowledge derived from the training data, a RAG workflow pulls relevant information and connects static LLMs with real-time data retrieval.

## Architecture

rag_16x9.avif

## Why do we need a RAG system?

Retrieval systems (RAG) give LLM systems access to factual, access-controlled, timely information.

1. RAG REDUCES HALLUCINATION
Example: In the financial services industry, providing accurate information on investment options is crucial because it directly impacts customers' purchasing decisions and financial well-being. RAG can help ensure that the information generated about stocks, bonds, or mutual funds

2. COST-EFFECTIVE ALTERNATIVE
Example: Banks often need to assess the creditworthiness of potential borrowers. Fine-tuning pre-trained language models to analyse credit histories can be resource-intensive. RAG architecture offers a cost-effective alternative by retrieving relevant financial data and credit history information from existing databases, combining this with pre-trained language models

3. CREDIBLE AND ACCURATE RESPONSES
Example: In customer support, providing accurate and helpful responses is essential for maintaining customer trust, as it demonstrates the company's commitment to providing reliable information and support. The RAG technique is able to do this very effectively by retrieving data from catalogues, policies, and past customer interactions to generate context-aware insights, ensuring that customers receive reliable information on product features, returns, and other inquiries.

4. DOMAIN-SPECIFIC INFORMATION
Example: In the legal industry, clients often require advice specific to their case or jurisdiction because different legal systems have unique rules and regulations, and understanding these nuances is crucial for effective legal representation. RAG can access domain-specific knowledge bases, such as local statutes and case law, to provide tailored information relevant to clients' legal needs.

https://www.advancinganalytics.co.uk/blog/2023/11/7/10-reasons-why-you-need-to-implement-rag-a-game-changer-in-ai

## RAG Practical Usecases

1. Document Question Answering Systems
2. Conversational agents
3. Real-time Event Commentary
4. Content Generation
5. Personalised Recommendation
6. Virtual Assistants

In [1]:
pip install -qU  langchain-google-genai
pip install -U langchain-text-splitters
pip install -qU langchain-community
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00


In [19]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY2')

In [20]:
import os
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

A RAG system can be divided into 3 stages:
1. Data Ingestion
2. Data Retreival
3. Data Generation/Synthesis

### Data Ingestion

In [7]:
from langchain_community.document_loaders import TextLoader

In [5]:
with open("state_of_the_union.txt", "r", encoding="utf8") as file:
  data = file.read()

In [8]:
loader = TextLoader("/content/state_of_the_union.txt", encoding='utf-8')

In [9]:
document = loader.load()

In [11]:
document[0].page_content

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

text_chunks = text_splitter.split_documents(document)

print(text_chunks[0].page_content)

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny.


In [15]:
print(len(text_chunks))

91


In [21]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = GoogleGenerativeAIEmbeddings(api_key=GEMINI_API_KEY, model="models/gemini-embedding-001")

In [25]:
vector_store = FAISS.from_documents(documents=text_chunks, embedding=embeddings)

In [26]:
retriever = vector_store.as_retriever()

In [31]:
from langchain_core.prompts import PromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

prompt = PromptTemplate(template=template)

In [28]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.2,
    max_output_tokens=1000
)

In [29]:
output_parser = StrOutputParser()

In [32]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [33]:
rag_chain.invoke("How is the United States supporting Ukraine economically and militarily?")

"The United States is supporting Ukraine economically and militarily in several ways. It is providing direct military and economic assistance to the Ukrainians in their fight for freedom. Over $1 billion in direct assistance has already been given to Ukraine.\n\nFurthermore, the U.S., along with twenty-seven members of the European Union and other countries such as the United Kingdom, Canada, and Japan, is enforcing powerful economic sanctions on Russia. These sanctions aim to inflict pain on Russia's economy and isolate it from the world. The United States also worked with 30 other countries to release 60 million barrels of oil from reserves to protect American businesses and consumers from the costs associated with these actions. These collective efforts aim to aid the Ukrainian people as they defend their country and to help ease their suffering."

In [34]:
rag_chain.invoke("What action is the U.S. taking to address rising gas prices?")

'To address rising gas prices, the U.S. is releasing 30 million barrels from its Strategic Petroleum Reserve. The United States has also collaborated with 30 other countries to release a total of 60 million barrels of oil from reserves worldwide. These actions are intended to help moderate gas prices domestically. The government is committed to using every tool available to protect American businesses and consumers from economic impacts, including those stemming from the Russian invasion of Ukraine. Additionally, there are efforts to promote electric vehicles, aiming to lower their prices so consumers can save money by avoiding gas pumps in the long term.'